<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Web-Scraping" data-toc-modified-id="Web-Scraping-1">Web Scraping</a></span><ul class="toc-item"><li><span><a href="#1.-Import-libraries" data-toc-modified-id="1.-Import-libraries-1.1">1. Import libraries</a></span></li><li><span><a href="#2.-Create-a-csv-file-for-the-data-we-want-to-scrap-and-a-data-frame" data-toc-modified-id="2.-Create-a-csv-file-for-the-data-we-want-to-scrap-and-a-data-frame-1.2">2. Create a csv file for the data we want to scrap and a data frame</a></span></li><li><span><a href="#3.-Web-scraping-with-Selenium" data-toc-modified-id="3.-Web-scraping-with-Selenium-1.3">3. Web scraping with Selenium</a></span></li></ul></li></ul></div>

# Web Scraping

We are going to do web scrapping on the Tripadvisor website to get the reviews from a restaurant. 

First, we will start with one restaurant(De kas) and if we have more time, will gather more information from more.

The purpose is to get a csv file with the reviews and other atrributes to be able to do a NLP (Natural Language processing) analysis with the reviews. 


Having a firts look at the site, we would like to extract:

    - Review Score
    - Title of the review
    - Review text
    - Date
    

I tried first with  BeautifulSoup and realized that is not enough, as when you are on the reviews you should press the more link to see the whole review, and not just the first lines. 
We need to launch a browser and control using Selenium, to simulate pressing the More link and executing the JavaScript code.

Another problem is that the score is graphical, with 5 green circles. We will need to expect further. 

In [14]:
#!pip install selenium

     |████████████████████████████████| 904 kB 3.5 MB/s eta 0:00:01


## 1. Import libraries

In [24]:
import csv
import time
from selenium import webdriver
import pandas as pd

## 2. Create a csv file for the data we want to scrap and a data frame

We could build a pandas data frame using each element as a key in a dictionary and a list as the values that we have obtain by scrapping. 

Anohter way is to use the library csv to create a csv file directly.

In [25]:
# Prepare an empty CSV file
csvFile = open("reviews.csv", "w", newline='', encoding="utf-8")
csvWriter = csv.writer(csvFile)
csvWriter.writerow(['Score','Date','Title','Review'])

## 3. Web scraping with Selenium 

In [64]:
#1.First page
url = "https://www.tripadvisor.nl/Restaurant_Review-g188590-d696917-Reviews-De_Kas-Amsterdam_North_Holland_Province.html"



We need to create a list for all the urls to get the reviews. 
The are 10 reviews displaied on each time. The first url in on the varialbe url. The rest we will create and store them on a list to iterate over it later on while scrapping. 

What it changes between url's, after the ifrst one is this part: -or10-, -or20-, etc.

There are around 1308 in english. The number on the url that changes displays the total number of reviews already seen in the previous URL's.

In [1]:
#range from 10 to 10, till the 1308 displayed.
numbers = list(range(10,1310, 10))

In [65]:
list_urls = [url]
for number in numbers:
    url_page= "https://www.tripadvisor.nl/Restaurant_Review-g188590-d696917--Reviews-or" + str(number) + "-De_Kas-Amsterdam_North_Holland_Province.html"
    list_urls.append(url_page)

In [67]:
len(list_urls)

131

In [ ]:
#2. we need to install webdriver.
#The Selenium API uses the WebDriver protocol to control a web browser

In [21]:
driver = webdriver.Chrome('/Users/lauratll/Documents/Data_Analysis/IronHack/Week_8/chromedriver')  

In [39]:
#it open a browser with it
driver.get(url)

<div class="prw_rup prw_reviews_text_summary_hsx" data-prwidget-name="reviews_text_summary_hsx" data-prwidget-init="handlers"><div class="entry"><p class="partial_entry">I read the incredible positive reviews and I had very high expectations.... unfortunately not met at all.
Hygiene and COVID...<span class="taLnk ulBlueLinks" onclick="widgetEvCall('handlers.clickExpand',event,this);">Meer</span></p></div></div>

In [72]:
from time import sleep
from random import randint

In [76]:

for url in list_urls:
    driver.get(url)
    # Find and click the More link (to load all reviews)
    driver.find_element_by_xpath("//span[@class='taLnk ulBlueLinks']").click()
    print(url)
    time.sleep(5) # Wait for reviews to load

    reviews = driver.find_elements_by_xpath("//div[@class='ui_column is-9']")
    num_page_items = min(len(reviews), 10)
    seconds_to_wait = randint(1,5)
    # Loop through the reviews found
    for i in range(num_page_items):
        # get the score, date, title and review
        try:
            score_class = reviews[i].find_element_by_xpath(".//span[contains(@class, 'ui_bubble_rating bubble_')]").get_attribute("class")
            score = score_class.split("_")[3]
            date = reviews[i].find_element_by_xpath(".//span[@class='ratingDate']").get_attribute("title")
            title = reviews[i].find_element_by_xpath(".//span[@class='noQuotes']").text
            review = reviews[i].find_element_by_xpath(".//p[@class='partial_entry']").text.replace("\n", "")
            # Save to CSV
            csvWriter.writerow((score, date, title, review))
        except: 
            continue
            

https://www.tripadvisor.nl/Restaurant_Review-g188590-d696917-Reviews-De_Kas-Amsterdam_North_Holland_Province.html
https://www.tripadvisor.nl/Restaurant_Review-g188590-d696917--Reviews-or10-De_Kas-Amsterdam_North_Holland_Province.html
https://www.tripadvisor.nl/Restaurant_Review-g188590-d696917--Reviews-or20-De_Kas-Amsterdam_North_Holland_Province.html
https://www.tripadvisor.nl/Restaurant_Review-g188590-d696917--Reviews-or30-De_Kas-Amsterdam_North_Holland_Province.html
https://www.tripadvisor.nl/Restaurant_Review-g188590-d696917--Reviews-or40-De_Kas-Amsterdam_North_Holland_Province.html
https://www.tripadvisor.nl/Restaurant_Review-g188590-d696917--Reviews-or50-De_Kas-Amsterdam_North_Holland_Province.html
https://www.tripadvisor.nl/Restaurant_Review-g188590-d696917--Reviews-or60-De_Kas-Amsterdam_North_Holland_Province.html
https://www.tripadvisor.nl/Restaurant_Review-g188590-d696917--Reviews-or70-De_Kas-Amsterdam_North_Holland_Province.html
https://www.tripadvisor.nl/Restaurant_Review-g

https://www.tripadvisor.nl/Restaurant_Review-g188590-d696917--Reviews-or680-De_Kas-Amsterdam_North_Holland_Province.html
https://www.tripadvisor.nl/Restaurant_Review-g188590-d696917--Reviews-or690-De_Kas-Amsterdam_North_Holland_Province.html
https://www.tripadvisor.nl/Restaurant_Review-g188590-d696917--Reviews-or700-De_Kas-Amsterdam_North_Holland_Province.html
https://www.tripadvisor.nl/Restaurant_Review-g188590-d696917--Reviews-or710-De_Kas-Amsterdam_North_Holland_Province.html
https://www.tripadvisor.nl/Restaurant_Review-g188590-d696917--Reviews-or720-De_Kas-Amsterdam_North_Holland_Province.html
https://www.tripadvisor.nl/Restaurant_Review-g188590-d696917--Reviews-or730-De_Kas-Amsterdam_North_Holland_Province.html
https://www.tripadvisor.nl/Restaurant_Review-g188590-d696917--Reviews-or740-De_Kas-Amsterdam_North_Holland_Province.html
https://www.tripadvisor.nl/Restaurant_Review-g188590-d696917--Reviews-or750-De_Kas-Amsterdam_North_Holland_Province.html
https://www.tripadvisor.nl/Resta

Selenium has the ability to grab elements by their ID, class, tag, or other properties. To find the ID, class, tag or other property you want to scrape, right click within Chrome browser and select Inspect (or you can press F12 to open the Inspector window).

In [ ]:
# Loop through the reviews found
for i in range(num_page_items):
    # get the score, date, title and review
    score_class = reviews[i].find_element_by_xpath(".//span[contains(@class, 'ui_bubble_rating bubble_')]").get_attribute("class")
    score = score_class.split("_")[3]
    date = reviews[i].find_element_by_xpath(".//span[@class='ratingDate']").get_attribute("title")
    title = reviews[i].find_element_by_xpath(".//span[@class='noQuotes']").text
    review = reviews[i].find_element_by_xpath(".//p[@class='partial_entry']").text.replace("\n", "")
    # Save to CSV
    csvWriter.writerow((score, date, title, review))

Now we create a loop to go trought all the pages, to get all the reviews that are displaied on the site 3o by 30 each time.

In [ ]:
for review in bsobj.findAll('a',{'class':'review_count'}):
  a = review['href']
  a = 'https://www.tripadvisor.in'+ a
  a = a[:(a.find('Reviews')+7)] + '-or{}' + a[(a.find('Reviews')+7):]
  print(a)
  links.append(a)